In [255]:
from splinter import Browser
from bs4 import BeautifulSoup as bs


In [256]:
import requests
import pandas as pd

In [257]:
#window users
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [258]:
url = 'https://www.zolo.ca/toronto-real-estate/houses'
browser.visit(url)

#print(soup.prettify())

In [259]:
# Iterate through all pages
results = []
for x in range(55):
    # HTML object
    
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = bs(html, 'html.parser')
    # Retrieve all link for listed houses
    list_items = soup.find_all("a", class_='address link-primary')

    # Iterate through each book
  
    for item in list_items:
        link = item['href'] 
        results.append(link)
    
    next_url = 'https://www.zolo.ca/toronto-real-estate/houses/page-' + str(x +1)
    request = requests.get(next_url)
    if request.status_code == 200:
        browser.visit(next_url)
    else:
        print("url colection compleated")
        break
    #print(next_url)

In [265]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
house_df = pd.DataFrame()
#'address', 'city', 'hoot', 'Type', 'Style, 'Size', 'Lot Size',
#'Age', 'Taxes', 'Walk Score', 'MLS®#', 'Days on Site', 'Listed By'
for link in results:
    browser.visit(link)
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = bs(html, 'html.parser')
    # Retrieve all elements that contain book information
    address= soup.find('h1', class_="address xs-text-2 sm-text-1 truncate bold").text
    area = soup.find('div', 'a', class_= "area xs-text-5 sm-text-4 truncate").text.split(", ")
    city = area[0]
    hoot = area[1]
    #house_type = soup.find('div', 'a', class_= "column-container sm-column-count-2 column-gap").text
    house_info = soup.find_all('div',  {'class':"column-container sm-column-count-2 column-gap"})
    #house_dl_text = []
    #house_list = [addres, city, hoot]
    house_dict={"address": address,"city" :city, "hoot": hoot}
    #house_df = pd.DataFrame(house_dict) 
    house_info_keys = []
    for i in house_info[0].find_all('dt'):
        house_info_keys.append(i.text)
        
    house_info_values = []
    for i in house_info[0].find_all('dd'):
        house_info_values.append(i.text.rstrip().lstrip())
    
    
    for key in house_info_keys:
        for value in house_info_values:
            house_dict[key] = value
            house_info_values.remove(value)
            break 
            
    house_df = house_df.append(house_dict, ignore_index=True)


In [266]:
house_df.head()

,Age,Days on Site,Listed By,Lot Size,MLS®#,Size,Style,Taxes,Type,Walk Score,address,city,hoot
0,0-5 years,"5 (Dec 11, 2019)","Zolo Realty, Brokerage",70.00 x 115 Feet,C4651523,5000 sqft,3-Storey,"$10,920 /yr",Detached,20,28 Caswell Drive,Toronto,Willowdale East
1,No Data,"20 (Nov 26, 2019)","Zolo Realty, Brokerage",33.00 x 120 Feet,E4642347,No Data,2-Storey,"$3,115 /yr",Detached,77,18 Vanbrugh Avenue,Toronto,Birchcliffe-cliffside
2,0-5 years,"26 (Nov 19, 2019)","Zolo Realty, Brokerage",32.00 x 107 Feet,E4638092,1500-2000 sqft,2-Storey,"$4,038 /yr",Detached,62,53 Northbrook Road,Toronto,East York
3,No Data,"1 (Dec 16, 2019)","Right At Home Realty Inc., Brokerage",57.50 x 143.93 Feet,W4654216,No Data,Bungalow,"$3,936 /yr",Detached,68,1023 Kipling Avenue,Toronto,Islington-city Centre West
4,No Data,"1 (Dec 16, 2019)","One Percent Realty Ltd., Brokerage",55.96 x 189.2 Feet,E4654229,No Data,2-Storey,"$5,215 /yr",Detached,45,82 Whistling Hills Drive,Toronto,Milliken
